In [2]:
# Import Packages
import numpy as np
import scipy.stats as stats
import os
import pickle
import scipy.io as scio
from ncdump_python3 import ncdump
import pickle
import Jacks_Functions as JF
import Area_Avg
import matplotlib
from matplotlib import pyplot as plt
from scipy.interpolate import griddata
from netCDF4 import Dataset
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

In [4]:
#Load in Dimensions
LLL = Dataset('LatLon.nc')
WACCM4_Lat = np.squeeze(LLL.variables['lat'])
WACCM4_Lon = np.squeeze(LLL.variables['lon'])
LLL.close()

Source_File = Dataset('Kernels/Interpolated/CAM3_Kernels.nc')

CAM3_Lat = np.squeeze(Source_File.variables['lat'])
CAM3_Lon = np.squeeze(Source_File.variables['lon'])

cx,cy = np.meshgrid(CAM3_Lon,CAM3_Lat)
wx,wy = np.meshgrid(WACCM4_Lon,WACCM4_Lat)

RCP45_PS = pickle.load(open('RCP45_Base_PS.pickle','rb'))
#interpolate the surface pressure to the CAM3 grid
RCP45_PS_interp = np.zeros([12,64,128])
for i in range(12):
    RCP45_PS_interp[i,:,:] = griddata(\
    (wx.flatten(),wy.flatten()),RCP45_PS[i,:,:].flatten(),\
    (cx,cy),method='linear')/100 #swap to hpa in the process
    
#Lists for iterating
Months = np.arange(0,12)
FP_Ensemble_Members = np.arange(5)

In [7]:
#Tile the plevs to match the shape of the kernel
CMIP_plevs_scalar = np.squeeze(Source_File.variables['lev'])

A = np.tile(CMIP_plevs_scalar.T,(CAM3_Lon.size,1))
B = np.transpose(A[:,:,None],(0,1,2))
C = np.tile(B,(1,1,CAM3_Lat.size))
D = np.transpose(C[:,:,:,None],(0,1,2,3))
E = np.tile(D,(1,1,1,12))
CMIP_plevs = np.swapaxes(E,3,0)
CMIP_plevs.shape

(12, 17, 64, 128)

In [8]:
#Calculate pressure level thickness using CMIP pressure levels and surface pressure
dp = np.zeros([12,17,64,128])

for i in range(12):
    for j in range(64):
        for k in range(128):
            dp[i,:,j,k] = JF.PlevThck(\
            PS = RCP45_PS_interp[i,j,k],plevs=CMIP_plevs_scalar,\
            p_top = min(CMIP_plevs_scalar))
            
dp = np.ma.masked_invalid(dp[:,:,:,:]/100) #CAM3 Kernel units are per 100 hPa

In [9]:
#estimation of the tropopause. 300hPA at the poles, raising
    #linearly to 100hpa at the tropics

p_tropopause_zonalmean_linear = np.zeros([64])

for y in range(64):
    if y <= len(CAM3_Lat)/2:
        if y == 0:
            p_tropopause_zonalmean_linear[y] = 300
        elif y == len(CAM3_Lat)/2:
            p_tropopause_zonalmean_linear[y] = 100
        else:
            p_tropopause_zonalmean_linear[y] = \
            p_tropopause_zonalmean_linear[y-1]\
            -(200/(len(CAM3_Lat)/2-1))
    else:
        if y == len(CAM3_Lat)/2:
            p_tropopause_zonalmean_linear[y] = 100
        elif y == len(CAM3_Lat):
            p_tropopause_zonalmean_linear[y] = 300
        else:
            p_tropopause_zonalmean_linear[y] = \
            p_tropopause_zonalmean_linear[y-1]\
            +(200/(len(CAM3_Lat)/2-1))
            
Q = np.tile(p_tropopause_zonalmean_linear.T,(CAM3_Lon.size,1))
R = np.transpose(Q[:,:,None],(0,1,2))
S = np.tile(R,(1,1,17))
T = np.transpose(S[:,:,:,None],(0,1,2,3))
U = np.tile(T,(1,1,1,12))
V = np.swapaxes(U,3,0)
p_tropopause_CAM3 = np.swapaxes(V,2,1)

#how does it look?
p_tropopause_CAM3.shape

(12, 17, 64, 128)

In [10]:
Kernel_CAM3_TA = np.ma.masked_greater(\
    np.squeeze(Source_File.variables['Ta_TOA']),1e10)
Kernel_CAM3_TA_clear = np.ma.masked_greater(\
    np.squeeze(Source_File.variables['Ta_TOA_CLR']),1e10)

Kernel_CAM3_TS = np.ma.masked_greater(\
    np.squeeze(Source_File.variables['Ts_TOA']),1e10)
Kernel_CAM3_TS_clear = np.ma.masked_greater(\
    np.squeeze(Source_File.variables['Ts_TOA_CLR']),1e10)

In [11]:
#climate change responses
RCP45_dST_CAM3Grid = pickle.load(open(\
   "Future Projection CC Responses/RCP45_dST_CAM3Grid.pickle","rb"),encoding='latin1')
RCP45_dST_CAM3Grid = np.swapaxes(RCP45_dST_CAM3Grid,2,3)
RCP45_dAT_CAM3Grid = pickle.load(open(\
    "Future Projection CC Responses/RCP45_dAT_CAM3Grid.pickle","rb"),encoding='latin1')
RCP45_dAT_CAM3Grid = np.swapaxes(RCP45_dAT_CAM3Grid,3,4)

In [13]:
#arrays for total temperature feedback
CAM3_RCP45_dLW_Ts = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_RCP45_dLW_Ta = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_RCP45_dTa_Tropo = np.zeros([FP_Ensemble_Members.size,12,17,64,128])
CAM3_RCP45_dLW_T = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_RCP45_T_EnergyB_GA = np.zeros([FP_Ensemble_Members.size])
CAM3_RCP45_T_EnergyB_AA = np.zeros([FP_Ensemble_Members.size])

for i in range(FP_Ensemble_Members.size):
    
    #Multiply surface temperature change by surface temp kernel
    CAM3_RCP45_dLW_Ts[i,:,:,:] = Kernel_CAM3_TS*RCP45_dST_CAM3Grid[i,:,:,:]
    
    #Mask out stratosphere air temp change values by changing them to zero
    CAM3_RCP45_dTa_Tropo[i,:,:,:,:] = \
    RCP45_dAT_CAM3Grid[i,:,:,:,:]*(CMIP_plevs>=p_tropopause_CAM3)
    
    #multiply air temp change and air temp kernel, then integrate
        #up to the tropopause
    CAM3_RCP45_dLW_Ta[i,:,:,:] = \
    np.ma.sum(Kernel_CAM3_TA*CAM3_RCP45_dTa_Tropo[i,:,:,:,:]*dp,1)
    
    # add the surface and air temperature feedback responses
    CAM3_RCP45_dLW_T[i,:,:,:] = CAM3_RCP45_dLW_Ta[i,:,:,:]+CAM3_RCP45_dLW_Ts[i,:,:,:]
    
    #Take the annual average and Arctic area average
    CAM3_RCP45_T_EnergyB_AA[i]  = np.mean(Area_Avg.LatLonavg_Time(\
            CAM3_RCP45_dLW_T[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon),axis=0)
    
    #Take the annual average and global area average
    CAM3_RCP45_T_EnergyB_GA[i]  = np.mean(Area_Avg.LatLonavg_Time(\
            CAM3_RCP45_dLW_T[i,:,:,:],CAM3_Lat[:],CAM3_Lon),axis=0)

In [14]:
print(CAM3_RCP45_T_EnergyB_GA)
print(CAM3_RCP45_T_EnergyB_AA)

[-3.63351626 -3.18142383 -3.47889802 -3.52133249 -3.38037785]
[-3.83170933 -2.96110921 -4.30640273 -4.07912035 -3.61181548]


In [15]:
#get feedback values from dividing by the surface temperature change
#Area average surface temperature change over 60-90N and globally
RCP45_dST_AA = np.zeros([FP_Ensemble_Members.size])
for i in range(FP_Ensemble_Members.size):    
    RCP45_dST_AA[i] = np.mean(Area_Avg.LatLonavg_Time(\
    RCP45_dST_CAM3Grid[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon[:]),axis=0)
    
CAM3_RCP45_T_Feedback_AA = CAM3_RCP45_T_EnergyB_AA/RCP45_dST_AA

In [16]:
print('Arctic Energy Budget',CAM3_RCP45_T_EnergyB_AA)
print('\n')
print('Arctic Feedback',CAM3_RCP45_T_Feedback_AA)

Arctic Energy Budget [-3.83170933 -2.96110921 -4.30640273 -4.07912035 -3.61181548]


Arctic Feedback [-1.80087235 -1.77599475 -1.61826275 -1.74074067 -1.65438782]
